In [4]:
import json
import joblib
import pickle
import pandas as pd
import numpy as np
import os
from datetime import datetime
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier



import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

import fairlearn

In [9]:
df=pd.read_csv('two-years-recid.csv')#.set_index('id')
df

,id,name,sex,dob,age,race,c_jail_in,c_jail_out,c_case_number,c_offense_date,...,r_case_number,r_offense_date,r_charge_degree,r_charge_desc,is_violent_recid,vr_case_number,vr_offense_date,vr_charge_degree,vr_charge_desc,two_year_recid
0,1,steven lux,Male,1953-06-15,62,Caucasian,2013-01-05 04:35:31,2013-01-07 03:18:03,13000208CF10A,2013-01-05,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0
1,2,andre small,Male,1987-10-01,28,African-American,2013-04-02 12:18:46,2013-04-04 07:54:22,13006354MM10A,2013-04-02,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0
2,3,willie gray,Male,1959-01-12,57,African-American,2014-10-31 12:02:01,2014-10-31 01:47:05,14040148MU10A,2014-10-30,...,15043364TC20A,2015-07-23,(M2),Driving License Suspended,0,NaN,NaN,NaN,NaN,1
3,4,nickson marcellus,Male,1996-07-11,19,African-American,2014-01-23 03:19:30,2014-01-23 01:04:34,13017969CF10A,NaN,...,16000241MM20A,2016-01-04,(M1),Possess Cannabis/20 Grams Or Less,0,NaN,NaN,NaN,NaN,1
4,5,patria barnes,Female,1978-06-06,37,Other,2013-12-08 01:55:28,2013-12-09 02:00:59,13022717MM10A,2013-12-07,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6109,6110,seccunda davis,Male,1987-05-13,28,African-American,2013-08-18 07:25:24,2013-08-19 09:01:42,13015644MM10A,2013-08-18,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0
6110,6111,mark montgomery,Male,1985-11-03,30,African-American,2013-03-23 01:32:34,2013-03-28 09:37:27,13005696MM10A,2013-03-23,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0
6111,6112,erica johnson,Female,1982-06-23,33,Caucasian,2013-09-29 09:25:30,2013-09-30 09:59:37,13013661CF10A,2013-09-29,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0
6112,6113,barry williams,Male,1988-04-22,27,African-American,2013-10-30 03:40:14,2013-12-07 01:53:45,13004112MM10A,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0


In [10]:
## REMOVING ROWS FOR GOAL

In [11]:
df = df.dropna(subset=['c_jail_out'])

# Suppose df_test has some different values, so we'll modify it directly
# Convert columns to datetime if they are not already
df['c_jail_in'] = pd.to_datetime(df['c_jail_in'])
df['c_jail_out'] = pd.to_datetime(df['c_jail_out'])

df['r_offense_date'] = pd.to_datetime(df['r_offense_date'])


# Calculate the difference between 'r_offense_date' and 'c_jail_out'
df['offense_jail_duration'] = (df['r_offense_date'] - df['c_jail_out']).dt.days

# Create a new column based on your conditions
df['two_year_recid'] = ((df['is_recid'] == 1) & (df['offense_jail_duration'] <= 365 * 2)).astype(int)

#
df['is_recid']=df['two_year_recid']

/tmp/ipykernel_3019/2141686846.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['c_jail_in'] = pd.to_datetime(df['c_jail_in'])
/tmp/ipykernel_3019/2141686846.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['c_jail_out'] = pd.to_datetime(df['c_jail_out'])
/tmp/ipykernel_3019/2141686846.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

In [12]:
df.count()

id                       5854
name                     5854
sex                      5854
dob                      5854
age                      5854
race                     5854
c_jail_in                5854
c_jail_out               5854
c_case_number            5854
c_offense_date           4983
c_arrest_date             871
c_charge_degree          5854
c_charge_desc            5848
juv_fel_count            5854
juv_misd_count           5854
juv_other_count          5854
priors_count             5854
compas_screening_date    5854
type_of_assessment       5854
decile_score             5854
score_text               5854
v_type_of_assessment     5854
v_decile_score           5854
v_score_text             5854
is_recid                 5854
r_case_number            2879
r_offense_date           2879
r_charge_degree          2879
r_charge_desc            2830
is_violent_recid         5854
vr_case_number            681
vr_offense_date           681
vr_charge_degree          681
vr_charge_

In [13]:
# Swap c_jail_in and c_jail_out where c_jail_out is before c_jail_in and the difference is less than one day
swap_mask = (df['c_jail_out'] < df['c_jail_in']) & ((df['c_jail_in'] - df['c_jail_out']).dt.days < 1)
df.loc[swap_mask, ['c_jail_in', 'c_jail_out']] = df.loc[swap_mask, ['c_jail_out', 'c_jail_in']].values


In [14]:
#non sense rows
df['invalid_jail_in_arrest'] = df['c_jail_in'] < df['c_arrest_date']
df['invalid_jail_in_offense'] = df['c_jail_in'] < df['c_offense_date']

# Remove rows where both conditions are true
df = df[~(df['invalid_jail_in_arrest'] & df['invalid_jail_in_offense'])]

# Drop the columns after filtering
df = df.drop(columns=['invalid_jail_in_arrest', 'invalid_jail_in_offense'])

/tmp/ipykernel_3019/2362044094.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['invalid_jail_in_arrest'] = df['c_jail_in'] < df['c_arrest_date']
/tmp/ipykernel_3019/2362044094.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['invalid_jail_in_offense'] = df['c_jail_in'] < df['c_offense_date']


In [15]:
df = df[df['c_jail_in'] >= '2012-01-01']

In [16]:
##FIL IN AVG
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
import joblib

# Define the categorize_charge function
# Frequency encoding
def frequency_encoding(df, column):
    freq_encoding = df[column].value_counts(normalize=True)
    df[f'{column}_freq'] = df[column].map(freq_encoding)
    return df

# Define the preprocess_data function
def preprocess_data(df):
    
    df['dob'] = pd.to_datetime(df['dob'])
    df['c_offense_date'] = pd.to_datetime(df['c_offense_date'])
    df['c_arrest_date'] = pd.to_datetime(df['c_arrest_date'])
    df['c_jail_in'] = pd.to_datetime(df['c_jail_in'])

    # Create unified_date
    df['unified_date'] = df['c_arrest_date'].combine_first(df['c_offense_date'])

    # Feature engineering
    df['age_birth'] = (pd.to_datetime('today') - df['dob']).dt.days // 365
    df['age_at_unified_date'] = (df['unified_date'] - df['dob']).dt.days // 365
    
    # Calculate averages for filling NaN values
    avg_time_offense_arrest = (df['c_arrest_date'] - df['c_offense_date']).dt.days.mean()
    avg_time_since_jail_in = (pd.to_datetime('today') - df['c_jail_in']).dt.days.mean()
    avg_time_to_jail = (df['c_jail_in'] - df['unified_date']).dt.days.mean()

    df['time_offense_arrest'] = (df['c_arrest_date'] - df['c_offense_date']).dt.days.fillna(avg_time_offense_arrest)  # Fill NaN with average
    df['time_since_jail_in'] = (pd.to_datetime('today') - df['c_jail_in']).dt.days.fillna(avg_time_since_jail_in)  # Duration in jail till today
    df['time_to_jail'] = (df['c_jail_in'] - df['unified_date']).dt.days.fillna(avg_time_to_jail)
    
    df['total_juv_crimes'] = df['juv_fel_count'] + df['juv_misd_count'] + df['juv_other_count']
    df['total_adult_crimes'] = df['priors_count'] - df['total_juv_crimes']

    # Apply frequency encoding and group less frequent categories
    df = frequency_encoding(df, 'c_charge_desc')

    # Create age bins and handle missing values
    bins = [18, 25, 35, 45, 55, 65, 100]
    labels = ['18-25', '26-35', '36-45', '46-55', '56-65', '66+']
    df['age_group'] = pd.cut(df['age_birth'], bins=bins, labels=labels, right=False)
    df['age_group'] = df['age_group'].cat.add_categories('missing_value').fillna('missing_value')
    
    # Extract more granular date features
    df['offense_month'] = df['c_offense_date'].dt.month.fillna(df['c_offense_date'].dt.month.mean())
    df['offense_day_of_week'] = df['c_offense_date'].dt.dayofweek.fillna(df['c_offense_date'].dt.dayofweek.mean())
    df['arrest_month'] = df['c_arrest_date'].dt.month.fillna(df['c_arrest_date'].dt.month.mean())
    df['arrest_day_of_week'] = df['c_arrest_date'].dt.dayofweek.fillna(df['c_arrest_date'].dt.dayofweek.mean())

    return df

# Load data (ensure df is loaded before this step)
# df = pd.read_csv('your_data.csv')  # Example loading method

# Apply preprocessing to the entire dataset
df = preprocess_data(df)

# Split data into train and test sets
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Define features
all_features = [
    "sex", 
    "race", 
    #"juv_fel_count", 
    #"juv_misd_count", 
    #"juv_other_count",
    "total_adult_crimes", 
    "c_charge_degree", 
   # "c_charge_desc", 
    "c_charge_desc_freq",
    "age_at_unified_date", 
    "total_juv_crimes",
    "offense_month", 
    "offense_day_of_week", 
    "arrest_month", 
    "arrest_day_of_week",
   # "age_group"
]

numerical_features = [
    #"juv_fel_count", 
    #"juv_misd_count", 
    #"juv_other_count", 
    "total_adult_crimes",
    #"time_offense_arrest", 
    #"time_since_jail_in", 
    "total_juv_crimes",
    "offense_month", 
    "offense_day_of_week", 
    "arrest_month", 
    "arrest_day_of_week",
    "age_at_unified_date", 
    #"age_birth",
    "c_charge_desc_freq"
]

categorical_features = [
    "sex", 
    "race", 
    "c_charge_degree", 
   # "c_charge_desc",
    #"age_group"
]

target = 'is_recid'

# Define preprocessing pipelines
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing_value')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Use the best model parameters found earlier
best_model_params = {
    'learning_rate': 0.01,
    'max_depth': 3,
    'n_estimators': 300,
    'subsample': 0.8,
    'random_state': 42
}

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier(**best_model_params))
])


# Train the updated pipeline
X_train = df_train[all_features]
y_train = df_train[target]
pipeline.fit(X_train, y_train)

# Evaluate the updated pipeline on the test set
X_test = df_test[all_features]
y_test = df_test[target]
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

# Print evaluation metrics
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-Score: {f1:.2f}')
print(f'ROC-AUC: {roc_auc:.2f}')
print(f'Confusion Matrix:\n{conf_matrix}')



Accuracy: 0.69
Precision: 0.68
Recall: 0.58
F1-Score: 0.63
ROC-AUC: 0.74
Confusion Matrix:
[[499 145]
 [219 308]]


In [36]:
#####RACE#######
from fairlearn.metrics import (
    demographic_parity_ratio,
    equalized_odds_ratio,
    true_positive_rate_difference,
    false_positive_rate_difference,
    selection_rate_difference
)

# Assuming y_test contains the true labels and y_pred contains the predicted labels
# Calculate fairness metrics
disparate_impact = demographic_parity_ratio(y_true=y_test, y_pred=y_pred, sensitive_features=X_test['race'])
equalized_odds = equalized_odds_ratio(y_true=y_test, y_pred=y_pred, sensitive_features=X_test['race'])
tpr_diff = true_positive_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test['race'])
fpr_diff = false_positive_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test['race'])
selection_rate_diff = selection_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test['race'])

print("Disparate Impact Ratio:", disparate_impact)
print("Equalized Odds Ratio:", equalized_odds)
print("True Positive Rate Difference:", tpr_diff)
print("False Positive Rate Difference:", fpr_diff)
print("Selection Rate Difference:", selection_rate_diff)


Disparate Impact Ratio: 0.2222222222222222
Equalized Odds Ratio: 0.0
True Positive Rate Difference: 1.0
False Positive Rate Difference: 0.29194630872483224
Selection Rate Difference: 0.5833333333333334


In [38]:
import numpy as np

# Calculate disparate impact ratio for each race group
def calculate_disparate_impact(y_true, y_pred, X_test, race_column):
    disparate_impact_ratios = {}
    
    # Mean of favorable outcomes in the overall dataset
    overall_favorable_outcomes = y_pred.mean()
    
    # Iterate over unique race values
    for race_value in X_test[race_column].unique():
        # Mask for the current race group
        race_mask = X_test[race_column] == race_value
        
        # Calculate favorable outcomes for the current race group
        favorable_outcomes_race = y_pred[race_mask].mean()
        
        # Calculate disparate impact ratio for the current race group
        if overall_favorable_outcomes == 0:
            disparate_impact_ratios[race_value] = np.nan  # Avoid division by zero
        else:
            disparate_impact_ratios[race_value] = favorable_outcomes_race / overall_favorable_outcomes
    
    return disparate_impact_ratios




In [39]:
# Call the function with your data
race_column = 'race'  # Replace with your race column name
disparate_impact_by_race = calculate_disparate_impact(y_test, y_pred, X_test, race_column)
print("Disparate Impact Ratios by Race Group:")
for race, ratio in disparate_impact_by_race.items():
    print(f"{race}: {ratio}")

Disparate Impact Ratios by Race Group:
African-American: 1.260811209439528
Asian: 0.43178466076696165
Caucasian: 0.6857756376887038
Other: 0.8469622191967325
Hispanic: 0.7951677911153947
Native American: 1.9430309734513274


In [37]:
#####SEX#######
from fairlearn.metrics import (
    demographic_parity_ratio,
    equalized_odds_ratio,
    true_positive_rate_difference,
    false_positive_rate_difference,
    selection_rate_difference
)

# Assuming y_test contains the true labels and y_pred contains the predicted labels
# Calculate fairness metrics
disparate_impact = demographic_parity_ratio(y_true=y_test, y_pred=y_pred, sensitive_features=X_test['sex'])
equalized_odds = equalized_odds_ratio(y_true=y_test, y_pred=y_pred, sensitive_features=X_test['sex'])
tpr_diff = true_positive_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test['sex'])
fpr_diff = false_positive_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test['sex'])
selection_rate_diff = selection_rate_difference(y_true=y_test, y_pred=y_pred, sensitive_features=X_test['sex'])

print("Disparate Impact Ratio:", disparate_impact)
print("Equalized Odds Ratio:", equalized_odds)
print("True Positive Rate Difference:", tpr_diff)
print("False Positive Rate Difference:", fpr_diff)
print("Selection Rate Difference:", selection_rate_diff)
#Disparate Impact Ratio: 0.6877922267679719
#Equalized Odds Ratio: 0.6940800300935722
#True Positive Rate Difference: 0.09593466043384635
#False Positive Rate Difference: 0.08354199571118559
#Selection Rate Difference: 0.14870037231636452

Disparate Impact Ratio: 0.5845301083396321
Equalized Odds Ratio: 0.49612086565945285
True Positive Rate Difference: 0.1357466063348416
False Positive Rate Difference: 0.12856175444079804
Selection Rate Difference: 0.17550021285653472


In [40]:
# Call the function with your data
race_column = 'sex'  # Replace with your race column name
disparate_impact_by_race = calculate_disparate_impact(y_test, y_pred, X_test, race_column)
print("Disparate Impact Ratios by Race Group:")
for race, ratio in disparate_impact_by_race.items():
    print(f"{race}: {ratio}")

Disparate Impact Ratios by Race Group:
Male: 1.0943507781507478
Female: 0.6396809789140172


In [68]:
## Serialization

In [74]:
# Serialization
TMP_DIR = ''

# Serialize the column names from the X DataFrame into a file named columns.json
# YOUR CODE HERE
with open(os.path.join(TMP_DIR, "columns.json"), 'w') as fh:
    json.dump(X_train.columns.tolist(), fh)

# Pickle the dtypes of the columns from the X DataFrame into a file named dtypes.pickle
# YOUR CODE HERE
with open(os.path.join(TMP_DIR,'dtypes.pickle'), 'wb') as fh:
    pickle.dump(X_train.dtypes, fh)
    
# Pickle the fitted pipeline into a file named pipeline.pickle
# YOUR CODE HERE
import joblib
joblib.dump(pipeline, os.path.join(TMP_DIR,'pipeline.pickle'))

# Baseline Scoring
roc_auc_score(y_test, y_pred)
#roc_auc_score(y_test, y_proba)

0.6804188716159676

In [ ]:
## Testing the App

In [5]:
import requests
import json

url = "http://127.0.0.1:5000/will_recidivate/"

data = {
    "id": "a1b2c345",
    "name": "John Doe",
    "sex": "Male",
    "dob": "1990-01-01",
    "race": "White",
    "juv_fel_count": 0,
    "juv_misd_count": 1,
    "juv_other_count": 0,
    "priors_count": 2,
    "c_case_number": "case1234",
    "c_charge_degree": "M",
    "c_charge_desc": "Theft",
    "c_offense_date": "2023-01-01",
    "c_arrest_date": "2023-01-02",
    "c_jail_in": "2023-01-03"
}

headers = {'Content-Type': 'application/json'}
r = requests.post(url, data=json.dumps(data), headers=headers)

print("Response status code:", r.status_code)
print("Response content:", r.text)

Response status code: 200
Response content: {
  "error": "Observation ID: \"a1b2c345\" already exists",
  "id": "a1b2c345",
  "outcome": false
}



In [29]:
#import pandas as pd
#print("My numpy version is: ", pd.__version__)

My numpy version is:  2.1.0


In [6]:
import requests

# Define the URL of your Flask application
url = 'http://127.0.0.1:5000/recidivism_result/'  # Update with your actual URL

# Define the observation data
data = {
    "id": "a1b2c345",
    "outcome": True
}

headers = {'Content-Type': 'application/json'}
response = requests.post(url, data=json.dumps(data), headers=headers)

# Check the response status code
if response.status_code == 200:
    try:
        result = response.json()
        observation_id = result.get('id')  # Correct key
        outcome = result.get('outcome')  # Correct key
        predicted_outcome = result.get('predicted_outcome')  # Correct key
        if observation_id is not None and outcome is not None:
            print("Recidivism outcome updated successfully for ID:", observation_id)
            print("True outcome:", outcome)
            print("Predicted outcome:", predicted_outcome)
        else:
            print("Observation ID or outcome not found in the response")
    except ValueError:
        print("Failed to parse JSON response")
else:
    print("Failed to update recidivism outcome:", response.text)


Recidivism outcome updated successfully for ID: a1b2c345
True outcome: True
Predicted outcome: False


In [8]:
import requests
import json

url = "https://capstone-production-9dd5.up.railway.app/will_recidivate/"

data = {
    "id": "a1b2c345",
    "name": "John Doe",
    "sex": "Male",
    "dob": "1990-01-01",
    "race": "White",
    "juv_fel_count": 0,
    "juv_misd_count": 1,
    "juv_other_count": 0,
    "priors_count": 2,
    "c_case_number": "case1234",
    "c_charge_degree": "M",
    "c_charge_desc": "Theft",
    "c_offense_date": "2023-01-01",
    "c_arrest_date": "2023-01-02",
    "c_jail_in": "2023-01-03"
}


headers = {'Content-Type': 'application/json'}
r = requests.post(url, data=json.dumps(data), headers=headers)

print("Response status code:", r.status_code)
print("Response content:", r.text)

Response status code: 502
Response content: <!doctype html><meta name=viewport content="width=device-width,initial-scale=1"><meta charset=utf-8><title>502 Bad Gateway</title><style media=screen>
      :root {
        --bg: hsl(250, 24%, 9%);
        --fg: hsl(0, 0%, 100%);
        --pink: hsl(270, 60%, 52%);
        --pink-600: hsl(270, 70%, 65%);
        --green-600: hsl(152, 38%, 60%);
        --red-600: hsl(1, 62%, 60%);
        --gray-100: hsl(246, 18%, 15%);
        --gray-500: hsl(246,  6%, 55%);
      }

      html,
      body {
        margin: 0;
        padding: 10px;
        height: 100%;
        overflow: hidden;
        background-color: var(--bg);
        color: var(--fg);
        font-family: -apple-system, BlinkMacSystemFont, Segoe UI, Roboto,
          Oxygen-Sans, Ubuntu, Cantarell, Helvetica Neue, sans-serif,
          Apple Color Emoji, Segoe UI Emoji, Segoe UI Symbol;
        line-height: 1.4em;
      }

      main {
        display: flex;
        flex-direction: co

In [4]:
import requests

def test_will_recidivate():
    url = "https://capstone-production-fe80.up.railway.app/will_recidivate/"
    data = {
        "id": "a1b2c3",
        "name": "John Doe",
        "sex": "Male",
        "dob": "1990-01-01",
        "race": "Caucasian",
        "juv_fel_count": 2,
        "juv_misd_count": 1,
        "juv_other_count": 0,
        "priors_count": 5,
        "c_case_number": "A123456",
        "c_charge_degree": "F",
        "c_charge_desc": "Burglary",
        "c_offense_date": "2022-01-01",
        "c_arrest_date": "2022-01-02",
        "c_jail_in": "2022-01-03"
    }

    try:
        response = requests.post(url, json=data)
        response.raise_for_status()  # Raise an error for bad status codes
        print(f"Status Code: {response.status_code}")
        print(response.json())
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        if response is not None:
            print(f"Status Code: {response.status_code}")
            print(response.text)

def test_recidivism_result():
    url = "https://capstone-production-fe80.up.railway.app/recidivism_result/"
    data = {
        "id": "a1b2c3",
        "outcome": True
    }

    try:
        response = requests.post(url, json=data)
        response.raise_for_status()  # Raise an error for bad status codes
        print(f"Status Code: {response.status_code}")
        print(response.json())
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        if response is not None:
            print(f"Status Code: {response.status_code}")
            print(response.text)

def list_db_contents():
    url = "https://capstone-production-fe80.up.railway.app/list-db-contents"

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad status codes
        print(f"Status Code: {response.status_code}")
        print(response.json())
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        if response is not None:
            print(f"Status Code: {response.status_code}")
            print(response.text)

# Run the tests
test_will_recidivate()
test_recidivism_result()
list_db_contents()


An error occurred: 500 Server Error: Internal Server Error for url: https://capstone-production-fe80.up.railway.app/will_recidivate/
Status Code: 500
<!doctype html>
<html lang=en>
  <head>
    <title>peewee.DataError: invalid input syntax for type boolean: &#34;A123456&#34;
LINE 1: ...&#34; WHERE ((&#34;t1&#34;.&#34;observation&#34; = &#39;c_case_number&#39;) = &#39;A123456&#39;)...
                                                             ^

 // Werkzeug Debugger</title>
    <link rel="stylesheet" href="?__debugger__=yes&amp;cmd=resource&amp;f=style.css">
    <link rel="shortcut icon"
        href="?__debugger__=yes&amp;cmd=resource&amp;f=console.png">
    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=debugger.js"></script>
    <script>
      var CONSOLE_MODE = false,
          EVALEX = false,
          EVALEX_TRUSTED = false,
          SECRET = "rtWfAJLTvW77osXwNf1d";
    </script>
  </head>
  <body style="background-color: #fff">
    <div class="debugger">
<h1>DataError<

Status Code: 200
{'id': 'a1b2c3', 'outcome': True, 'predicted_outcome': False}
Status Code: 200
[{'id': 1, 'observation': '{"id": "9U0k3IlW0Lfc6GY1Yp9dWA", "name": "6cea57c2fb6cbc2a40411135005760f241fffc3e5e67ab99882726431037f908", "sex": "Male", "dob": "1990-01-01", "race": "Caucasian", "juv_fel_count": 2, "juv_misd_count": 1, "juv_other_count": 0, "priors_count": 5, "c_case_number": "A123456", "c_charge_degree": "F", "c_charge_desc": "Burglary", "c_offense_date": "2022-01-01", "c_arrest_date": "2022-01-02", "c_jail_in": "2022-01-03"}', 'observation_id': '9U0k3IlW0Lfc6GY1Yp9dWA', 'outcome': False, 'predicted_outcome': False}, {'id': 2, 'observation': '{"id": "a1b2c3", "name": "6cea57c2fb6cbc2a40411135005760f241fffc3e5e67ab99882726431037f908", "sex": "Male", "dob": "1990-01-01", "race": "Caucasian", "juv_fel_count": 2, "juv_misd_count": 1, "juv_other_count": 0, "priors_count": 5, "c_case_number": "A123456", "c_charge_degree": "F", "c_charge_desc": "Burglary", "c_offense_date": "2022-01

In [5]:
curl -X POST https://capstone-production-fe80.up.railway.app/will_recidivate/ -d '{"id": "test123",
        "name": "John Doe",
        "sex": "Male",
        "dob": "1990-01-01",
        "race": "Caucasian",
        "juv_fel_count": 2,
        "juv_misd_count": 1,
        "juv_other_count": 0,
        "priors_count": 5,
        "c_case_number": "A123456",
        "c_charge_degree": "F",
        "c_charge_desc": "Burglary",
        "c_offense_date": "2022-01-01",
        "c_arrest_date": "2022-01-02",
        "c_jail_in": "2022-01-03"}' -H "Content-Type:application/json"

SyntaxError: unterminated string literal (detected at line 1) (710772726.py, line 1)

In [ ]:
curl -X POST https://capstone-production-d66f.up.railway.app/recidivism_result -d '{
         "id": "123456abc",
        "outcome": true
         }' -H "Content-Type:application/json"